<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">KSO Tutorials #10: Run machine learning models on footage</h1>
<h3 align="right">Written by the KSO Team</h3>

# Set up and requirements

### Install and import Python packages

In [ ]:
from IPython.display import clear_output
import os
import sys

try:
    # Enable external widgets
    from google.colab import output

    output.enable_custom_widget_manager()

    IN_COLAB = True
    print("Running in Colab...")

    # Clone repo
    !git clone --recurse-submodules -b master https://github.com/ocean-data-factory-sweden/kso.git
    %pip install -qr <(sed '/Pillow/d;/ipywidgets/d' kso/yolov5_tracker/requirements.txt) -qr <(sed '/Pillow/d;/ipywidgets/d' kso/yolov5_tracker/yolov5/requirements.txt) -qr <(sed '/Pillow/d;/ipywidgets/d' kso/kso_utils/requirements.txt)

    # Fix libmagic issue
    !apt-get -qq update && apt-get -qq install -y libmagic-dev > /dev/null

    # Navigate to the correct folder
    os.chdir("kso/tutorials")

except:
    IN_COLAB = False


# Ensure widgets are shown properly
!jupyter nbextension enable --user --py widgetsnbextension
!jupyter nbextension enable --user --py jupyter_bbox_widget
!jupyter nbextension enable --user --py ipysheet

clear_output()
if IN_COLAB == True:
    print("Running in Colab: All packages are installed and ready to go!")
else:
    print("Running locally... you're good to go!")

In [ ]:
# Set the directory of the libraries
import sys, os
from pathlib import Path
from datetime import date

# Enables testing changes in utils
%load_ext autoreload
%autoreload 2

# Specify the path of the tutorials
sys.path.append("..")

# Enable third-party widgets(ipysheet)
from google.colab import output

output.enable_custom_widget_manager()

# Import required modules
import kso_utils.tutorials_utils as t_utils
import kso_utils.project_utils as p_utils
import kso_utils.t8_utils as t8

print("Packages loaded successfully")

### Choose your project

In [ ]:
project_name = kso_widgets.choose_project()

In [ ]:
project = p_utils.find_project(project_name=project_name.value)

In [ ]:
# Initialise pp
pp = ProjectProcessor(project)

In [ ]:
# Initialise mlp
mlp = MLProjectProcessor(pp)

# Run model on custom footage

### Choose the model

In [ ]:
model = mlp.choose_model()

### Choose folder to download the model

In [ ]:
download_dir = kso_widgets.choose_folder(".", "where to download the model")

In [ ]:
# @title <font size="5">↓ ឵឵឵឵<i>Download model</font> { vertical-output: true }
artifact_dir = mlp.get_model(model.value, download_dir.selected)

In [ ]:
# @title <font size="5">↓ ឵឵឵឵<i>Choose your custom footage</font> { vertical-output: true }
source = kso_widgets.choose_footage(
    project,
    pp.db_info,
    project.movie_folder if project.movie_folder not in [None, "None"] else ".",
    "custom footage",
)

In [ ]:
source_value = t_utils.process_source(source)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Choose folder where to save the runs</font> { vertical-output: true }
# This should be left as default value in most cases.
save_dir = kso_widgets.choose_folder(".", "runs output")

In [ ]:
# @title <font size="5">↓ ឵឵<i>Choose a confidence threshold for evaluation</font> { vertical-output: true }
conf_thres = t_utils.choose_conf()

In [ ]:
# @title <font size="5">↓ ឵឵<i>Run model over selected custom footage</font> { vertical-output: true }
mlp.detect_yolov5(
    source=source_value,
    save_dir=save_dir.selected,
    conf_thres=conf_thres.value,
    artifact_dir=artifact_dir,
)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Choose folder with the evaluation data</font> { vertical-output: true }
eval_dir = kso_widgets.choose_folder(
    save_dir.selected
    if "save_dir" in vars() and save_dir.selected is not None
    else ".",
    "runs output",
)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Add the data to Weights and Biases</font> { vertical-output: true }
mlp.save_detections_wandb(conf_thres.value, model.value, eval_dir.selected)

In [ ]:
# @title <font size="5">↓ ឵឵<i>View model output</font> { vertical-output: true }
viewer = y_utils.choose_files(eval_dir.selected)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Investigate training and validation datasets (only image data)</font> { vertical-output: true }
train_dataset, val_dataset = mlp.get_dataset(model.value)

In [ ]:
# Training set
y_utils.get_data_viewer(os.path.join(train_dataset, "data/images"))

In [ ]:
# Validation set
y_utils.get_data_viewer(os.path.join(val_dataset, "data/images"))

# Track unique individuals (Optional)

In [ ]:
mlp.track_individuals(
    source=source_value,
    artifact_dir=artifact_dir,
    eval_dir=eval_dir.selected,
    conf_thres=conf_thres.value,
    img_size=(540, 540),
)

In [ ]:
# END